In [5]:
from covid import utils
import numpy as np
import pandas as pd

In [57]:
path = "results/GA_20210508185341"
gen = 0

individuals_path =  f"{path}/individuals/individuals_{gen}.pkl"
score_path       =  f"{path}/final_scores/final_score_{gen}.pkl"
overview_path    =  f"{path}/generation_overview.csv"

final_score = utils.read_pickle(score_path)
individuals = utils.read_pickle(individuals_path)
overview = pd.read_csv(overview_path)

In [58]:
overview.sort_values('mean_score')

,generation,individual,mean_score
0,0,2544933905168,4365.723017
1,0,2544933906608,5357.139245


In [60]:
for i in sorted(individuals, key=lambda x: x.mean_score):
    print(f"ID: {i.ID}")
    print(f"Score: {i.mean_score}")
    print("Strategy count:")
    print(f"{'U':>5}{'D':>5}{'N':>5}")
    for w in range(1,5):
        print(f"{w}: {i.strategy_count[gen]['U'][w]:<5}{i.strategy_count[gen]['D'][w]:<5}{i.strategy_count[gen]['N'][w]:<5}")
    print(f"Genes:\n{i.genes}")
    print()

ID: 2544933905168
Score: 4365.723017107147
Strategy count:
    U    D    N
1: 0    2    2    
2: 2    7    3    
3: 8    4    2    
4: 4    0    0    
Genes:
[[[0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]]

 [[0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]]

 [[0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]]]

ID: 2544933906608
Score: 5357.139244825748
Strategy count:
    U    D    N
1: 0    2    2    
2: 2    7    3    
3: 8    4    2    
4: 4    0    0    
Genes:
[[[1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]]

